In [8]:

import numpy as np
import pandas as pd 
import networkx as nx
import matplotlib.pyplot as plt
import pandapower as pp 
import math


import matplotlib.gridspec as gridscpec


pd.set_option('display.float_format', lambda x:'{:.4f}'.format(x))

plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update(
    {
        'font.size':10,
        "text.usetex": False,
        "font.family": "serif",
        "font.sans-serif": ['Computer Modern'],
    }
)

#Definiendo parámetros de potencia P [MW] y Q [MVA]
P=150 
Q=100

#crear una red vacía
net = pp.create_empty_network()

#crear barras
bus1 = pp.create_bus(net, vn_kv=110, name="Barra 1")
bus2 = pp.create_bus(net, vn_kv=220, name="Barra 2")
bus1a = pp.create_bus(net, vn_kv=220, name="Barra 1A")
bus2a = pp.create_bus(net, vn_kv=220, name="Barra 2A")
bus3a = pp.create_bus(net, vn_kv=220, name="Barra 3A")
bus1b = pp.create_bus(net, vn_kv=220, name="Barra 1B")
bus2b = pp.create_bus(net, vn_kv=220, name="Barra 2B")


#crear trafo entre bus 1 y bus 2
pp.create_transformer(net, hv_bus=bus2, lv_bus=bus2, std_type="100 MVA 220/110 kV")

#crear el generador en bus1 y definirlo como la barra slack
pp.create_ext_grid(net, bus1, vm_pu=1.0, name="Barra Slack")

#crear cargas entre barras
pp.create_load(net, bus1a, p_mw=0.2*P, q_mvar=0.2*Q, name="Carga 1A")
pp.create_load(net, bus2a, p_mw=0.35*P, q_mvar=0.35*Q, name="Carga 2A")
pp.create_load(net, bus3a, p_mw=0.15*P, q_mvar=0.15*Q, name="Carga 3A")
pp.create_load(net, bus1b, p_mw=0.1*P, q_mvar=0.1*Q, name="Carga 1B")
pp.create_load(net, bus2b, p_mw=0.6*P, q_mvar=0.6*Q, name="Carga 2B")

#crear linea entre barras
pp.create_line(net, from_bus=bus2, to_bus=bus1a, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L21A', in_service=True)
pp.create_line(net, from_bus=bus1a, to_bus=bus2a, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L1A2A', in_service=True)
pp.create_line(net, from_bus=bus2a, to_bus=bus3a, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L2A3A', in_service=True)
pp.create_line(net, from_bus=bus3a, to_bus=bus2b, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L3A2B', in_service=True)
pp.create_line(net, from_bus=bus2, to_bus=bus1b, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L21B', in_service=True)
pp.create_line(net, from_bus=bus1b, to_bus=bus2b, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L1B2B', in_service=True)

pp.runpp(net)



